In [ ]:
%%capture
!pip install -q "gradio==4.44.0" "reportlab==4.0.9" "Pillow==10.3.0

In [ ]:
import os, tempfile, re
from datetime import datetime
from decimal import Decimal, ROUND_HALF_UP

import gradio as gr
from PIL import Image

from reportlab.lib.pagesizes import A4
from reportlab.lib.units import mm
from reportlab.pdfgen import canvas
from reportlab.lib import colors
from reportlab.platypus import Table, TableStyle, Paragraph
from reportlab.lib.styles import ParagraphStyle
from reportlab.lib.enums import TA_LEFT


# =========================
# Ajustes de layout
# =========================
MARGIN_LEFT   = 15 * mm
MARGIN_RIGHT  = 15 * mm
MARGIN_TOP    = 12 * mm
MARGIN_BOTTOM = 15 * mm

FONT_MAIN      = "Helvetica"
FONT_BOLD      = "Helvetica-Bold"
SIZE_BASE      = 9
SIZE_SMALL     = 8
SIZE_HEADER    = 10
SIZE_TITLE     = 12
SIZE_TOTAL_BIG = 16

# Anchos de columnas de la tabla
W_DESC  = 112 * mm   # Concepto
W_QTY   = 22  * mm   # Cantidad
W_PRICE = 28  * mm   # Precio
W_AMT   = 28  * mm   # Total por ítem


# =========================
# Utilidades de número y texto
# =========================
def dec(x):
    return Decimal(str(x))

def money_dec(x):
    return dec(x).quantize(Decimal("0.01"), rounding=ROUND_HALF_UP)

def fmt_pesos_apostrophe(value, decimals=0):
    """
    Formato tipo CO del ejemplo: 1'280.000 (por defecto sin decimales).
    """
    value = dec(value)
    if decimals == 0:
        s = f"{int(value):,}"          # 1,280,000
        s = s.replace(",", "'")        # 1'280'000
        parts = s.split("'")
        if len(parts) >= 2:
            s = "'".join(parts[:-1]) + "." + parts[-1]  # 1'280.000
        return s
    else:
        s = f"{value:,.{decimals}f}".replace(",", "'")
        if "." in s:
            i = s.rfind("'")
            if i != -1 and i < s.find("."):
                s = s[:i] + "." + s[i+1:]
        return s

def fmt_number(x, decimals=0):
    return fmt_pesos_apostrophe(x, decimals=decimals)

def clean_text(s: str) -> str:
    """Quita caracteres no imprimibles y normaliza espacios."""
    if not isinstance(s, str):
        s = str(s)
    import re as _re
    s = _re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f]", "", s)
    s = _re.sub(r"[ \t]+", " ", s).strip()
    return s


# =========================
# Colores (HEX -> ReportLab) y paleta desde logo
# =========================
def hex_to_rgb_float(hx):
    if not hx:
        return (0, 0, 0)
    hx = hx.strip().lstrip('#')
    if len(hx) == 3:
        hx = ''.join([c*2 for c in hx])
    r = int(hx[0:2], 16) / 255.0
    g = int(hx[2:4], 16) / 255.0
    b = int(hx[4:6], 16) / 255.0
    return (r, g, b)

def lighten(rgb, factor=0.75):
    r, g, b = rgb
    return (min(1.0, r + (1-r)* (1-factor)),
            min(1.0, g + (1-g)* (1-factor)),
            min(1.0, b + (1-b)* (1-factor)))

def darken(rgb, factor=0.85):
    r, g, b = rgb
    return (r*factor, g*factor, b*factor)

def dominant_color_from_logo(logo_path):
    try:
        with Image.open(logo_path) as im:
            im = im.convert("RGBA")
            bg = Image.new("RGBA", im.size, (255,255,255,255))
            bg.paste(im, mask=im.getchannel("A"))
            im = bg.convert("RGB")
            small = im.resize((64, 64))
            pal = small.quantize(colors=8, method=2)
            palette = pal.getpalette()
            counts = pal.getcolors()
            counts = sorted(counts, key=lambda x: x[0], reverse=True)
            best = (0,0,0)
            for cnt, idx in counts:
                r = palette[3*idx]
                g = palette[3*idx+1]
                b = palette[3*idx+2]
                if (r,g,b) != (255,255,255):
                    best = (r/255.0, g/255.0, b/255.0)
                    break
            return best
    except Exception:
        pass
    return hex_to_rgb_float("#F35A1F")


# =========================
# Parser de ítems
# =========================
def parse_items(text):
    items, subtotal, errores = [], Decimal("0"), []
    for idx, line in enumerate((text or "").strip().splitlines(), start=1):
        raw = line.strip()
        if not raw:
            continue
        parts = [p.strip() for p in raw.split("|")]
        if len(parts) != 3:
            errores.append(f"Línea {idx}: formato inválido (usa: Concepto|Cantidad|Precio)")
            continue
        desc, qty_txt, price_txt = parts
        qty_txt   = qty_txt.replace(",", ".")
        price_txt = price_txt.replace(",", ".")
        try:
            qty   = dec(qty_txt)
            price = dec(price_txt)
        except:
            errores.append(f"Línea {idx}: cantidad/precio no numéricos")
            continue
        if qty <= 0 or price < 0:
            errores.append(f"Línea {idx}: cantidad>0 y precio≥0")
            continue
        amount = qty * price
        subtotal += amount
        items.append({"desc": clean_text(desc), "qty": qty, "price": price, "amount": amount})
    return items, subtotal, errores


# =========================
# Generador del PDF
# =========================
def generar_pdf_remision(datos, logo_file=None, use_logo_palette=False,
                         hex_primary="#F35A1F", hex_secondary="#222222"):
    if use_logo_palette and logo_file and os.path.exists(logo_file):
        primary_rgb = dominant_color_from_logo(logo_file)
    else:
        primary_rgb = hex_to_rgb_float(hex_primary)
    secondary_rgb = hex_to_rgb_float(hex_secondary)
    primary_light = lighten(primary_rgb, 0.75)
    primary_dark  = darken(primary_rgb, 0.85)

    tmpdir = tempfile.mkdtemp()
    file_path = os.path.join(tmpdir, f"{datos['numero'] or 'REMISION'}.pdf")

    c = canvas.Canvas(file_path, pagesize=A4)
    W, H = A4
    left  = MARGIN_LEFT
    right = W - MARGIN_RIGHT
    top   = H - MARGIN_TOP
    y     = top

    # Banda superior + logo
    band_h = 16 * mm
    c.setFillColor(colors.Color(*primary_rgb))
    c.rect(left, H - MARGIN_TOP - band_h, right - left, band_h, stroke=0, fill=1)

    x_text = left + 2*mm
    if logo_file and os.path.exists(logo_file):
        try:
            from reportlab.lib.utils import ImageReader
            img = ImageReader(logo_file)
            iw, ih = img.getSize()
            max_h, max_w = band_h - 4*mm, 40*mm
            scale = min(max_w/iw, max_h/ih)
            lw, lh = iw*scale, ih*scale
            c.drawImage(img, left + 2*mm, H - MARGIN_TOP - (band_h - lh)/2 - lh,
                        width=lw, height=lh, preserveAspectRatio=True, mask='auto')
            x_text = left + 2*mm + lw + 6*mm
        except:
            pass

    c.setFillColor(colors.white)
    c.setFont(FONT_BOLD, SIZE_TITLE)
    c.drawString(x_text, H - MARGIN_TOP - band_h/2 + 3, clean_text((datos["empresa"]["nombre"] or "").upper()))
    c.setFillColor(colors.black)

    y = H - MARGIN_TOP - band_h - 6 * mm

    # Datos empresa
    c.setFont(FONT_BOLD, SIZE_HEADER)
    c.setFillColor(colors.Color(*primary_dark))
    c.drawString(left, y, "DATOS DE LA EMPRESA")
    c.setFillColor(colors.black)
    y -= 5 * mm
    c.setFont(FONT_MAIN, SIZE_BASE)
    c.drawString(left, y, clean_text(f"NIT: {datos['empresa'].get('nit','')}"));       y -= 4.5 * mm
    c.drawString(left, y, clean_text(f"Dirección: {datos['empresa'].get('direccion','')}")); y -= 4.5 * mm
    c.drawString(left, y, clean_text(f"Correo: {datos['empresa'].get('correo','')}   Teléfono: {datos['empresa'].get('telefono','')}"))
    y -= 6 * mm

    # Caja remisión
    box_w, box_h = 90 * mm, 20 * mm
    box_x = right - box_w
    box_y = y + 8 * mm
    c.setLineWidth(1.2)
    c.setStrokeColor(colors.Color(*primary_rgb))
    c.roundRect(box_x, box_y, box_w, box_h, 6, stroke=1, fill=0)
    c.setFont(FONT_BOLD, SIZE_TITLE)
    c.setFillColor(colors.Color(*primary_rgb))
    c.drawCentredString(box_x + box_w / 2, box_y + box_h - 7 * mm, "REMISIÓN DE MERCANCÍA")
    c.setFillColor(colors.black)
    c.setFont(FONT_MAIN, SIZE_BASE)
    c.drawString(box_x + 4 * mm, box_y + 5 * mm, clean_text(f"N°: {datos['numero'] or '-'}"))
    c.drawRightString(box_x + box_w - 4 * mm, box_y + 5 * mm, clean_text(f"Fecha de emisión: {datos['fecha']}"))

    # Datos cliente
    y -= 2 * mm
    c.setFont(FONT_BOLD, SIZE_HEADER)
    c.setFillColor(colors.Color(*primary_dark))
    c.drawString(left, y, "DATOS DEL CLIENTE")
    c.setFillColor(colors.black)
    y -= 5 * mm
    c.setFont(FONT_MAIN, SIZE_BASE)
    c.drawString(left, y, clean_text(f"Nombre: {datos['cliente'].get('nombre','')}"));      y -= 4.5 * mm
    c.drawString(left, y, clean_text(f"Documento: {datos['cliente'].get('documento','')}")); y -= 4.5 * mm
    c.drawString(left, y, clean_text(f"Dirección: {datos['cliente'].get('direccion','')}")); y -= 4.5 * mm
    c.drawString(left, y, clean_text(f"Teléfono: {datos['cliente'].get('telefono','')}"))
    y -= 6 * mm

    c.setLineWidth(0.8)
    c.setStrokeColor(colors.Color(*primary_rgb))
    c.line(left, y, right, y)
    y -= 3.5 * mm

    # Tabla
    data = [["Concepto", "Cantidad", "Precio", "Total"]]
    for it in datos["items"]:
        data.append([
            it["desc"],
            fmt_number(it["qty"], 2),
            fmt_number(it["price"], 0),
            fmt_number(it["amount"], 0),
        ])

    header_bg = colors.Color(*lighten(primary_rgb, 0.65))
    header_tx = colors.black

    table = Table(
        data,
        colWidths=[W_DESC, W_QTY, W_PRICE, W_AMT],
        rowHeights=None
    )
    table.setStyle(TableStyle([
        ("FONT", (0,0), (-1,0), FONT_BOLD, SIZE_BASE),
        ("BACKGROUND", (0,0), (-1,0), header_bg),
        ("TEXTCOLOR", (0,0), (-1,0), header_tx),
        ("LINEABOVE", (0,0), (-1,0), 0.8, colors.Color(*primary_rgb)),
        ("LINEBELOW", (0,0), (-1,0), 0.8, colors.Color(*primary_rgb)),
        ("FONT", (0,1), (-1,-1), FONT_MAIN, SIZE_BASE),
        ("VALIGN", (0,0), (-1,-1), "MIDDLE"),
        ("ALIGN", (1,1), (-1,-1), "RIGHT"),
        ("ALIGN", (0,0), (0,-1), "LEFT"),
        ("LINEBELOW", (0,-1), (-1,-1), 0.4, colors.Color(*primary_rgb)),
    ]))

    try:
        table_w, table_h = table.wrapOn(c, right - left, A4[1])
        table_x = left
        table_y = y - table_h
        table.drawOn(c, table_x, table_y)
        y = table_y - 4 * mm
    except Exception as e:
        raise RuntimeError(f"Error al dibujar la tabla: {e}. Revisa anchos de columnas y textos.")

    # Forma de pago
    c.setFont(FONT_BOLD, SIZE_HEADER)
    c.setFillColor(colors.Color(*darken(primary_rgb, 0.85)))
    c.drawString(left, y, "Forma de pago:")
    c.setFillColor(colors.black)
    c.setFont(FONT_MAIN, SIZE_BASE)
    c.drawString(left + 30 * mm, y, clean_text(datos.get("forma_pago", "") or ""))
    y -= 8 * mm

    # Total en caja
    total_txt = fmt_number(datos["total"], 0)
    box_t_w, box_t_h = 55 * mm, 12 * mm
    box_t_x = right - box_t_w
    box_t_y = y - box_t_h + 2*mm
    c.setFillColor(colors.Color(*primary_rgb))
    c.roundRect(box_t_x, box_t_y, box_t_w, box_t_h, 4, stroke=0, fill=1)
    c.setFillColor(colors.white)
    c.setFont(FONT_BOLD, SIZE_BASE)
    c.drawString(box_t_x + 3*mm, box_t_y + 3.5*mm, "TOTAL")
    c.setFont(FONT_BOLD, SIZE_TOTAL_BIG)
    c.drawRightString(box_t_x + box_t_w - 3*mm, box_t_y + 3.5*mm, total_txt)
    c.setFillColor(colors.black)
    y -= 14 * mm

    # Firmas
    c.setFont(FONT_BOLD, SIZE_BASE)
    c.setFillColor(colors.Color(*darken(primary_rgb, 0.85)))
    c.drawString(left, y, "ENTREGA CONFORME")
    c.drawString(left + (right-left)/2 + 2*mm, y, "RECIBE CONFORME")
    c.setFillColor(colors.black)
    y -= 16

    mid_x = left + (right - left) / 2
    c.setStrokeColor(colors.Color(*primary_rgb))
    c.line(left, y, left + 60*mm, y)
    c.line(mid_x + 2*mm, y, mid_x + 2*mm + 60*mm, y)
    y -= 4.5 * mm
    c.setFont(FONT_MAIN, SIZE_SMALL)
    c.drawString(left, y, "Nombre y Firma del Transportador/Despachador")
    c.drawString(mid_x + 2*mm, y, "Nombre y Firma del Cliente/Receptor")
    y -= 7 * mm

    c.setFont(FONT_BOLD, SIZE_BASE)
    c.drawString(left, y, "C.C.:")
    c.drawString(left + 45*mm, y, "Fecha y Hora de Entrega:")
    c.drawString(mid_x + 2*mm, y, "C.C.:")
    c.drawString(mid_x + 2*mm + 45*mm, y, "Fecha y Hora de Recibido:")
    y -= 10 * mm

    # Condiciones
    cond_title = "CONDICIONES DE ENTREGA Y GARANTÍA"
    cond_text = datos.get("condiciones_texto", "") or (
        "1. Recepción y Conformidad:\n"
        "- El cliente o receptor debe revisar la mercancía antes de firmar.\n"
        "- La firma en \"Recibe Conforme\" certifica que los productos se entregaron completos y sin defectos físicos visibles.\n"
        "- No se aceptarán reclamos por daños físicos o estéticos luego de firmada la remisión.\n"
        "2. Uso de la Garantía:\n"
        "- La garantía cubre únicamente defectos de fabricación (5 años para el colchón, 1 año para la base) y no aplica por mal uso, derrames o limpieza inadecuada.\n"
        "- Para hacer efectiva la garantía, es indispensable presentar esta remisión."
    )

    style_title = ParagraphStyle("cond_t", fontName=FONT_BOLD, fontSize=SIZE_HEADER, leading=12, alignment=TA_LEFT, textColor=colors.Color(*darken(primary_rgb, 0.85)))
    style_text  = ParagraphStyle("cond_p", fontName=FONT_MAIN, fontSize=SIZE_SMALL, leading=12, alignment=TA_LEFT)

    P_title = Paragraph(clean_text(cond_title), style_title)
    w, h = P_title.wrapOn(c, right-left, A4[1])
    P_title.drawOn(c, left, y - h)
    y -= h + 2 * mm

    P = Paragraph(clean_text(cond_text).replace("\n", "<br/>"), style_text)
    w, h = P.wrapOn(c, right-left, A4[1])
    if y - h < MARGIN_BOTTOM:
        c.showPage()
        y = A4[1] - MARGIN_TOP
    P.drawOn(c, left, y - h)
    y -= h

    c.showPage()
    c.save()
    return file_path


# =========================
# Procesamiento del formulario
# =========================
def procesar_formulario(
    # Empresa
    emp_nombre, emp_nit, emp_dir, emp_tel, emp_mail,
    # Cliente
    cli_nombre, cli_doc, cli_dir, cli_tel,
    # Documento
    numero, fecha, forma_pago, items_texto,
    # Estilo
    logo_file, use_logo_palette, hex_primary, hex_secondary
):
    items, subtotal, errores = parse_items(items_texto)
    total = subtotal

    datos = {
        "empresa": {
            "nombre": emp_nombre, "nit": emp_nit, "direccion": emp_dir,
            "telefono": emp_tel, "correo": emp_mail
        },
        "cliente": {
            "nombre": cli_nombre, "documento": cli_doc,
            "direccion": cli_dir, "telefono": cli_tel
        },
        "numero": numero or "RM-000000",
        "fecha":  fecha or datetime.now().strftime("%d-%m-%Y"),
        "items":  items,
        "forma_pago": forma_pago,
        "subtotal": float(subtotal),
        "total": float(total),
    }

    # Resolver ruta del logo si se subió
    logo_path = None
    if logo_file:
        if isinstance(logo_file, str) and os.path.exists(logo_file):
            logo_path = logo_file
        elif isinstance(logo_file, dict):
            p = logo_file.get("name")
            if p and os.path.exists(p):
                logo_path = p
            else:
                data = logo_file.get("data")
                if data:
                    tmpdir = tempfile.mkdtemp()
                    logo_path = os.path.join(tmpdir, "logo.png")
                    with open(logo_path, "wb") as f:
                        f.write(data if isinstance(data, (bytes, bytearray)) else data.read())
        else:
            p = getattr(logo_file, "name", None)
            if p and os.path.exists(p):
                logo_path = p

    pdf_path = generar_pdf_remision(
        datos,
        logo_file=logo_path,
        use_logo_palette=use_logo_palette,
        hex_primary=hex_primary or "#F35A1F",
        hex_secondary=hex_secondary or "#222222"
    )

    resumen = f"TOTAL: {fmt_number(total, 0)}\n"
    if errores:
        resumen += "\nAdvertencias:\n- " + "\n- ".join(errores)

    return (
        resumen,
        gr.update(
            value=pdf_path,
            visible=True,
            label=f"Descargar PDF ({os.path.basename(pdf_path)})",
            interactive=True
        )
    )


# =========================
# Wrapper seguro para mostrar traceback
# =========================
import traceback

def procesar_formulario_safe(*args, **kwargs):
    try:
        resumen, download_update = procesar_formulario(*args, **kwargs)
        if not isinstance(resumen, str):
            resumen = str(resumen)
        if not isinstance(download_update, dict):
            download_update = gr.update(visible=False)
        return resumen, download_update
    except Exception:
        tb = traceback.format_exc()
        print("===== ERROR EN procesar_formulario =====\n", tb)
        return f"⚠️ Se produjo un error:\n\n{tb}", gr.update(visible=False)


# =========================
# Interfaz Gradio
# =========================
with gr.Blocks(title="Remisión (colores, formas y logo)") as demo:
    gr.Markdown("## 🧾 Remisión de mercancía — Con colores del ejemplo y logo")
    with gr.Row():
        with gr.Column():
            gr.Markdown("### Datos de la empresa")
            emp_nombre = gr.Textbox(label="Nombre legal", value="COLCHONES GANESHA")
            emp_nit    = gr.Textbox(label="NIT", value="901 XXX XXX - X")
            emp_dir    = gr.Textbox(label="Dirección", value="Calle 18 # 11 Este 84 Sur, Mosquera, Cundinamarca")
            emp_tel    = gr.Textbox(label="Teléfono", value="322-913-7616")
            emp_mail   = gr.Textbox(label="Correo", value="jclivingstore23@gmail.com")
        with gr.Column():
            gr.Markdown("### Datos del cliente")
            cli_nombre = gr.Textbox(label="Nombre")
            cli_doc    = gr.Textbox(label="Documento (C.C./NIT)")
            cli_dir    = gr.Textbox(label="Dirección")
            cli_tel    = gr.Textbox(label="Teléfono")

    with gr.Row():
        with gr.Column():
            gr.Markdown("### Documento")
            numero     = gr.Textbox(label="N° Remisión", value="RM-202601-01236")
            fecha      = gr.Textbox(label="Fecha de emisión (DD-MM-AAAA)",
                                    value=datetime.now().strftime("%d-%m-%Y"))
            forma_pago = gr.Textbox(label="Forma de pago", value="")
        with gr.Column():
            gr.Markdown("### Ítems (Concepto|Cantidad|Precio)")
            items_texto = gr.Textbox(
                lines=7,
                value="COMBO SUPER PILLOW 1.40 X 1.90|1|1280000\nobsequio 2 almohadas y protector|1|0",
                placeholder="Ej.: Concepto|Cantidad|Precio"
            )

    gr.Markdown("### Estilo (colores y logo)")
    with gr.Row():
        logo_file = gr.File(label="Logo (imagen)", file_types=["image"])
        use_logo_palette = gr.Checkbox(value=True, label="Usar paleta automática del logo")
    with gr.Row():
        hex_primary   = gr.Textbox(label="Color primario (HEX)", value="#F35A1F", info="Ej.: #F35A1F")
        hex_secondary = gr.Textbox(label="Color secundario/texto (HEX)", value="#222222")

    btn = gr.Button("Calcular y generar PDF", variant="primary")
    resumen = gr.Textbox(label="Resumen", interactive=False)
    download_btn = gr.DownloadButton(label="Descargar PDF", visible=False)

    btn.click(
        fn=procesar_formulario_safe,
        inputs=[emp_nombre, emp_nit, emp_dir, emp_tel, emp_mail,
                cli_nombre, cli_doc, cli_dir, cli_tel,
                numero, fecha, forma_pago, items_texto,
                logo_file, use_logo_palette, hex_primary, hex_secondary],
        outputs=[resumen, download_btn]
    )

# En Colab, activa debug y share
demo.launch(debug=True, show_error=True, share=True)
